# Component Demonstration Notebook

A notebook to demonstrate some of the features of the Component class

Author: gavin.treseder@essentialenergy.com.au

In [1]:
#  Add root folder to python path
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

import numpy as np 
import pandas as pd

import plotly.express as px

from pof.component import Component

# remove alter
from helper import fill_blanks

In [2]:
comp = Component().set_demo()

In [3]:
comp.mc_timeline(t_end=100, n_iterations=100)

100%|██████████| 100/100 [00:02<00:00, 47.70it/s]


In [4]:

def update_pof_figure():
    pof = comp.expected_pof(t_end=200)

    df = pd.DataFrame(pof)
    df.index.name = 'time'
    df = df.reset_index().melt(id_vars = 'time', var_name='source', value_name='pof')

    fig = px.line(df,x='time', y='pof', color='source')
    return fig

update_pof_figure().show()

In [40]:
def expected_inspection_interval(x_min, x_max, n_iterations=100):
    rc = dict()
    comp.reset()

    for i in range(max(1, x_min),x_max,1):

        # Set t_interval
        for fm in comp.fm.values():
            if 'inspection' in list(fm.tasks):
                fm.tasks['inspection'].t_interval = i
        
        comp.mc_timeline(t_end=100, n_iterations=n_iterations)

        rc[i] = comp.expected_risk_cost_df().groupby(by=['task'])['cost'].sum()
        rc[i]['inspection_interval'] = i

        # Reset component
        comp.reset()

    df = pd.DataFrame().from_dict(rc, orient='index')
    df['total_cost'] = df.drop(['inspection_interval', 'risk'], axis=1).sum(axis=1)
    df['total'] = df['total_cost'] + df['risk']
    
    return df

df = expected_inspection_interval(0, 10)

100%|██████████| 100/100 [00:00<00:00, 211.86it/s]


In [6]:

def update_inspection_interval_figure():
    
    df = expected_inspection_interval(0,10)
    df_plot = df.melt(id_vars='inspection_interval', var_name = 'source', value_name='cost')
    fig = px.line(df_plot, x="inspection_interval", y="cost", color='source', title='Risk v Cost at different Inspection Intervals')
    return fig

update_inspection_interval_figure().show()

100%|██████████| 10/10 [00:00<00:00, 81.30it/s]


In [41]:
comp= Component().set_demo()

df_mean = expected_inspection_interval(0, 10, 1000)

comp= Component().set_demo()

for fm in comp.fm.values():
    fm.cof.risk_cost_total = 12000

df_low_pof = expected_inspection_interval(0, 10, 1000)

comp= Component().set_demo()

for fm in comp.fm.values():
    fm.cof.risk_cost_total = 120000

df_low_pof = expected_inspection_interval(0, 10, 1000)

comp= Component().set_demo()

for fm in comp.fm.values():
    fm.failure_dist.alpha = fm.failure_dist.alpha * 2
    fm.calc_init_dist()

df_low_pof = expected_inspection_interval(0, 10, 1000)

100%|██████████| 1000/1000 [00:04<00:00, 242.19it/s]


In [7]:
df['total'] = df['total'] - df['total'].min()
df['cost_total'] = df[]

SyntaxError: invalid syntax (<ipython-input-7-d5a85fe54ec6>, line 2)

In [44]:
df_lowc = df_high.copy()

In [45]:
for fm in comp.fm.values():
    fm.cof.risk_cost_total = 120000

df_high = expected_inspection_interval(0, 10, 1000)

100%|██████████| 1000/1000 [00:14<00:00, 70.93it/s]


In [49]:
    df_plot = df_lowc.melt(id_vars='inspection_interval', var_name = 'source', value_name='cost')
    fig = px.line(df_plot, x="inspection_interval", y="cost", color='source', title='Risk v Cost at different Inspection Intervals')
    fig.show()

In [77]:
df_all = pd.concat(dict(
    mean = df_mean,
    high_c = df_high,
    low_c = df_lowc,
    low_pof = df_low_pof,
), ignore_index=False).reset_index()

df_all = df_all.rename(columns = {'level_0':'scenario'})



df_plot = df_all#df_all.melt(id_vars='inspection_interval', var_name = 'source', value_name='cost')
fig = px.line(df_plot, x="inspection_interval", y="total_cost", color='scenario', title='Risk v Cost at different Inspection Intervals')
fig.show()

In [76]:
df_all

,cm,inspection,on_condition_repair,risk,inspection_interval,total_cost,total
scenario,,,,,,,
0,1115,14850,169,9750,1,16134,25884
1,1090,7497,209,10050,2,8796,18846
2,1230,4950,208,11350,3,6388,17738
3,1510,3747,204,14400,4,5461,19861
4,2100,2997,186,20000,5,5283,25283
5,3130,2400,164,30500,6,5694,36194
6,3810,2100,153,36550,7,6063,42613
7,3979,1800,143,38850,8,5922,44772
8,4742,1650,114,46350,9,6506,52856


In [8]:
erc = comp.expected_risk_cost()

In [9]:
df = comp.expected_risk_cost_df()
df

,failure_mode,task,time,cost,cost_cumulative
0,random,inspection,0,0,0
0,random,inspection,1,0,0
0,random,inspection,2,0,0
0,random,inspection,3,0,0
0,random,inspection,4,0,0
...,...,...,...,...,...
11,fast_aging,risk,96,0,0
11,fast_aging,risk,97,0,0
11,fast_aging,risk,98,0,0
11,fast_aging,risk,99,0,0


In [10]:
comp.fm['random'].expected_cost_df()

,time,task,cost,cost_cumulative
0,0,inspection,0.0,0.0
1,1,inspection,0.0,0.0
2,2,inspection,0.0,0.0
3,3,inspection,0.0,0.0
4,4,inspection,0.0,0.0
...,...,...,...,...
195,195,cm,0.0,0.0
196,196,cm,0.0,0.0
197,197,cm,0.0,0.0
198,198,cm,0.0,0.0


In [11]:
fig = px.area(
    df,
    x="time",
    y="cost_cumulative",
    color="task",
    line_group = 'failure_mode'
)
fig

101

In [31]:
comp.mc_timeline(200, n_iterations=1000)

100%|██████████| 1000/1000 [00:09<00:00, 106.56it/s]


In [33]:

import plotly.graph_objects as go


def update_condition_fig(*args):
    """ Updates the condition figure"""

    fig = go.Figure()

    ecl = comp.expected_condition_loss()

    cmap = px.colors.qualitative.Safe
    ci = 0

    for cond_name, cond in ecl.items():
        # Format the data for plotting
        length = len(cond['mean'])
        time = np.linspace(0,length  -1, length, dtype=int) # TODO take time as a variable
        x = np.append(time,time[::-1])
        y = np.append(cond['upper'], cond['lower'][::-1])

        # Add the boundary
        fig.add_trace(go.Scatter(
            x=x,
            y=y,
            fill='toself',
            fillcolor='rgba' + cmap[ci][3:-2] + ',0.2)',
            line_color='rgba(255,255,255,0)',
            showlegend=False,
            name=cond_name,
        ))
        fig.add_trace(go.Scatter(
            x=time, y=cond['mean'],
            line_color=cmap[ci],
            name=cond_name,
        ))

        ci = ci + 1

        fig.update_traces(mode='lines')
        fig.update_xaxes(title_text='Time')
        fig.update_yaxes(title_text='Condition (%)')

    return fig


fig = update_condition_fig()
fig.show()

In [11]:
fm = comp.fm['random']
stdev = 1

expected = dict()
for cond_name, condition in fm.conditions.items():

    ec = np.array([fm._timelines[x][cond_name] for x in fm._timelines])

    mean = condition.perfect - ec.mean(axis=0)
    sd = ec.std(axis=0)
    upper = mean + sd*stdev
    lower = mean - sd*stdev

    upper[upper > condition.perfect] = condition.perfect
    lower[lower < condition.failed] = condition.failed

    expected[cond_name] = dict(
        lower=lower,
        mean=mean,
        upper=upper,
        sd=sd,
            )

TypeError: 'numpy.float64' object does not support item assignment

In [18]:
fm.timeline

{}

In [13]:
n_time = 1001
time = list(np.linspace(0,n_time - 1, n_time))

failure_modes = []
tasks = []
times = []
costs = []
cumulative_costs = []

n_fm = len(erc)

for fm, fm_values in erc.items():

    for task, task_values in fm_values.items():
        failure_mode = np.full(n_time, fm)
        task = np.full(n_time, task)
        cost = np.full(n_time, 0)
        cost[task_values['time']] = task_values['cost']
        cumulative_cost = cost.cumsum()

        failure_modes = np.append(failure_modes, failure_mode)
        tasks = np.append(tasks, task)
        costs = np.append(costs, cost)
        times = np.append(times, time)
        cumulative_costs = np.append(cumulative_costs, cumulative_cost)


df = pd.DataFrame(
    dict(
        failure_mode=failure_modes,
        task=tasks,
        time=times,
        cost=costs,
        cumulative_cost=cumulative_costs,
    )
)

IndexError: arrays used as indices must be of integer (or boolean) type

In [14]:
times = np.linspace(0,100,101, dtype=int)
n=101

time = np.array(df['time'].values.tolist())
cost = np.array(df['cost'].values.tolist())
fmt = np.repeat(df[['failure_mode', 'task']].to_numpy(),n, axis=0)
pd.DataFrame(np.column_stack((a, vals.ravel())), columns=df.columns)

NameError: name 'a' is not defined

In [15]:
profile = comp.fm['random'].expected_cost()
profile['risk'] = comp.fm['random'].expected_risk()

AttributeError: 'FailureMode' object has no attribute 'expected_cost'

In [16]:
%timeit comp.fm['slow_aging'].expected_cost_df()

27.6 ms ± 9.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
%timeit comp.fm['slow_aging'].expected_risk_cost_df()

3.79 ms ± 535 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
rc = comp.fm['slow_aging'].expected_risk_cost()

In [19]:
new_index = pd.Index(np.arange(0, 200, 1), name="time")

t = rc['inspection']['time']
c = rc['inspection']['cost']

In [20]:
tc = dict(task=[], time=[], cost=[])

for k, v in rc.items():
    tc['task'] = np.append(tc['task'], np.full(len(v['time']), k))
    for m in ['time', 'cost']:
        time = tc[m]
        
        

        tc[m] = np.append(tc[m], v[m])

array([ 0. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. ,
       50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. ,
       50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. ,
       50. , 50. , 50. , 50. , 50. , 50. , 50. , 49.5])

In [21]:
df = comp.fm['random'].expected_risk_cost_df()
df.index.name = 'task'
df = df.reset_index()

In [22]:
df = comp.fm['slow_aging'].expected_risk_cost_df()

In [23]:

px.line(
    df.reset_index(),
    x="time",
    y="cumulative_cost",
    color="task",
    line_group =
)

SyntaxError: invalid syntax (<ipython-input-23-cdf092dcd93a>, line 7)